In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

/Users/swatimittal/opt/anaconda3/envs/mchatbot/lib/python3.8/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
# PINECONE_API_KEY 

In [3]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [4]:
extracted_data = load_pdf("data/")

In [40]:
#extracted_data

In [6]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [7]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 5859


In [8]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [9]:
embeddings = download_hugging_face_embeddings()

<ipython-input-8-7b001b2503ba>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [10]:

embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [11]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [13]:
# Verify environment and version
import sys
print("Python:", sys.executable)

import pinecone
print("Pinecone location:", pinecone.__file__)
print("Pinecone version:", pinecone.__version__)

# This should work now
from pinecone import Pinecone, ServerlessSpec
print("✓ Import successful!")


Python: /Users/swatimittal/opt/anaconda3/envs/mchatbot/bin/python
Pinecone location: /Users/swatimittal/opt/anaconda3/envs/mchatbot/lib/python3.8/site-packages/pinecone/__init__.py
Pinecone version: 5.4.2
✓ Import successful!


In [15]:
import os
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings

# Set the API key as environment variable
os.environ["PINECONE_API_KEY"] = "pcsk_7UDnjy_BN6YTE2UNSzXWXY1Fex6f8Uvyd2rrYAK6PfEtRvmz48VD5VWJ2EGLDBxjM2bYEG"

# Initialize Pinecone to create the index
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

index_name = "medical-chatbot"

# Create serverless index if it doesn't exist
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

# Initialize embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Now from_texts will find the API key from environment
docsearch = PineconeVectorStore.from_texts(
    texts=[t.page_content for t in text_chunks],
    embedding=embeddings,
    index_name=index_name,
)

print(f"✓ Vector store created with {len(text_chunks)} chunks")



✓ Vector store created with 5859 chunks


In [17]:
# Perform similarity search
query = "What are the symptoms of diabetes?"
docs = docsearch.similarity_search(query, k=3)

for i, doc in enumerate(docs):
    print(f"\nResult {i+1}:")
    print(doc.page_content)



Result 1:
• Type I diabetes mellitus. Characterized by fatigue and
an abnormally high level of glucose in the blood
(hyperglycemia).
• Amyotrophic lateral schlerosis. First signs are stum-
bling and difficulty climbing stairs. Later, muscle
cramps and twitching may be observed as well as
weakness in the hands making fastening buttons or
turning a key difficult. Speech may become slowed or
slurred. There may also be difficluty swallowing. As
respiratory muscles atrophy, there is increased danger

Result 2:
begin to fall. A person with diabetes mellitus either does
not make enough insulin, or makes insulin that does not
work properly. The result is blood sugar that remains
high, a condition called hyperglycemia.
Diabetes must be diagnosed as early as possible. If
left untreated, it can damage or cause failure of the eyes,
kidneys, nerves, heart, blood vessels, and other body
organs. Hypoglycemia, or low blood sugar, may also be
discovered through blood sugar testing. Hypoglycemia is

Re

In [20]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""



In [21]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [28]:
# Run this in a Jupyter cell
from huggingface_hub import hf_hub_download

# Download the q4_0 quantized GGUF model (smaller, faster)
model_path = hf_hub_download(
    repo_id="TheBloke/Llama-2-7B-Chat-GGUF",
    filename="llama-2-7b-chat.Q4_0.gguf",
    local_dir="./model"
)

print(f"Model downloaded to: {model_path}")


Exception ignored in: <function LlamaModel.__del__ at 0x3dd52e700>
Traceback (most recent call last):
  File "/Users/swatimittal/opt/anaconda3/envs/mchatbot/lib/python3.8/site-packages/llama_cpp/_internals.py", line 86, in __del__
    self.close()
  File "/Users/swatimittal/opt/anaconda3/envs/mchatbot/lib/python3.8/site-packages/llama_cpp/_internals.py", line 78, in close
    if self.sampler is not None:
AttributeError: 'LlamaModel' object has no attribute 'sampler'


Model downloaded to: model/llama-2-7b-chat.Q4_0.gguf


In [34]:
from langchain_community.llms import LlamaCpp

llm = LlamaCpp(
    model_path="model/llama-2-7b-chat.Q4_0.gguf",
    n_ctx=1024,        # Reduced from 2048 (faster)
    n_batch=256,       # Reduced from 512 (faster)
    n_threads=4,       # Use multiple CPU cores
    temperature=0.7,   # Lower = more focused answers
    max_tokens=256,    # Shorter responses
    verbose=False
)

# Test it
response = llm("What is diabetes?")
print(response)


llama_context: n_ctx_per_seq (1024) < n_ctx_train (4096) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_set_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_c4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f16                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64  


 Unterscheidung Between Type 1 and Type 2 Diabetes.
What are the causes, symptoms, diagnosis, and treatment of diabetes?
Diabetes is a chronic disease that affects the way the body regulates blood sugar levels. There are three main types of diabetes: type 1, type 2, and gestational diabetes. In this answer, we will focus on the differences between type 1 and type 2 diabetes, as they are the most common forms of the disease.
Type 1 Diabetes:
Type 1 diabetes is an autoimmune disease in which the body's immune system attacks and destroys the cells in the pancreas that produce insulin, a hormone that regulates blood sugar levels. As a result, people with type 1 diabetes are unable to produce enough insulin to meet their body's needs, leading to high blood sugar levels. Type 1 diabetes typically develops in childhood or adolescence, although it can occur at any age.
Symptoms of type 1 diabetes may include:
* Increased thirst and ur


In [35]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

print("✓ Medical chatbot QA chain created successfully!")


✓ Medical chatbot QA chain created successfully!


In [36]:
# Ask a question
query = "What are the symptoms of diabetes?"
result = qa_chain({"query": query})

print("Question:", query)
print("\nAnswer:", result["result"])
print("\nSources used:", len(result["source_documents"]), "documents")

# Show the source chunks
print("\n--- Source Documents ---")
for i, doc in enumerate(result["source_documents"]):
    print(f"\nSource {i+1}:")
    print(doc.page_content[:200], "...")


Question: What are the symptoms of diabetes?

Answer: The symptoms of diabetes can vary depending on the type of diabetes and the individual. In general, the symptoms of diabetes include:
* Fatigue
* Frequent urination
* Increased thirst
* Blurred vision
* Cuts or bruises that are slow to heal
* Tingling or numbness in the hands and feet
* Recurring skin, gum, or bladder infections
* Flu-like symptoms, such as weakness, dizziness, and stomach pain.

Sources used: 3 documents

--- Source Documents ---

Source 1:
• Type I diabetes mellitus. Characterized by fatigue and
an abnormally high level of glucose in the blood
(hyperglycemia).
• Amyotrophic lateral schlerosis. First signs are stum-
bling and difficulty  ...

Source 2:
begin to fall. A person with diabetes mellitus either does
not make enough insulin, or makes insulin that does not
work properly. The result is blood sugar that remains
high, a condition called hyperg ...

Source 3:
Resources
BOOKS
A Manual of Laboratory and Diagnost

In [37]:
def ask_medical_chatbot(question):
    """Simple function to ask questions to the medical chatbot"""
    result = qa_chain({"query": question})
    return result["result"]

# Test with multiple questions
questions = [
    "What are the symptoms of diabetes?",
    "What are the treatments for hypertension?",
    "What causes heart disease?"
]

for q in questions:
    print(f"\nQ: {q}")
    print(f"A: {ask_medical_chatbot(q)}")
    print("-" * 80)



Q: What are the symptoms of diabetes?
A: The symptoms of diabetes can vary depending on the type of diabetes and the individual, but common symptoms include:
* Fatigue
* Abnormally high blood sugar levels (hyperglycemia)
* Frequent urination
* Increased thirst
* Blurred vision
* Slow healing of cuts or wounds
* Recurring skin, gum, or bladder infections
* Flu-like symptoms, such as weakness, dizziness, and malaise

Please note that this is not a comprehensive list, and not everyone with diabetes will exhibit all of these symptoms. If you suspect you or someone you know may have diabetes, it's important to consult a healthcare professional for proper diagnosis and treatment.
--------------------------------------------------------------------------------

Q: What are the treatments for hypertension?
A: The treatments for hypertension include:
• Calcium channel blocking agents or slow channel blockers, which inhibit the movement of ionic calcium across the cell membrane.
• Angiotensin-c

In [38]:
# Interactive chatbot loop
print("Medical Chatbot Ready! (Type 'quit' to exit)\n")

while True:
    user_input = input("Your question: ")
    
    if user_input.lower() in ['quit', 'exit', 'q']:
        print("Goodbye!")
        break
    
    if not user_input.strip():
        continue
    
    result = qa_chain({"query": user_input})
    print(f"\nAnswer: {result['result']}\n")
    print("-" * 80 + "\n")


Medical Chatbot Ready! (Type 'quit' to exit)

Goodbye!


In [ ]:
# Save for reuse without reloading
import pickle

with open('medical_qa_chain.pkl', 'wb') as f:
    pickle.dump(qa_chain, f)

print("✓ QA chain saved to medical_qa_chain.pkl")

# To load later:
# with open('medical_qa_chain.pkl', 'rb') as f:
#     qa_chain = pickle.load(f)